# SABR and the volatility smile calibration
#### Roland Grinis - MIPT & GrinisRIT

This is a stochastic volatility model that describes the dynamics of the forward directly in the risk neutral measure:

$$
\begin{split}
\text{d}f_t & =  \sigma_t f_t^\beta \text{d}W_t^1   \\
\text{d}\sigma_t & =  \upsilon \sigma_t \left( \rho \text{d}W_t^1 + \sqrt{1-\rho^2} \text{d}W_t^2 \right) 
\end{split}
$$

where $-1 < \rho < 1$ is the vol correlation, $0 \leq \beta \leq 1$ is the CIR exponent and the Wiener processes are independent.

In [2]:
import torch
from torch.utils.cpp_extension import load
import numpy as np
import numba as nb
import matplotlib.pyplot as plt
plt.style.use("dark_background")
%matplotlib inline
torch.manual_seed(987654);

In [3]:
!mkdir -p build

In [4]:
sabr = load(name='sabr',
             build_directory='./build',
             sources=['sabr.cc'],
             extra_cflags=['-Wall -Wextra -Wpedantic -O3 -std=c++17'],
             verbose=False)

In [5]:
a = torch.randn(5, dtype=torch.float64)

In [7]:
sabr.test(a)

tensor([-0.3602, -1.0800, -0.8943, -0.5205, -0.7527], dtype=torch.float64)

## 1. Implied Volatility

Zelen & Severo approximation for the standard normal CDF $\text{N}(x) = \frac{1}{\sqrt{2\pi}}\int^x_{-\infty} \text{d}z \exp(-z^2/2)$ is given by: 
$$
\text{N}(x) = 1 - \text{N}'(x) \sum_{i=1}^{5} b_i t^i + \epsilon (x)
$$
where $t = (1 + b_0 x)^{-1}$, the error is bounded by $|\epsilon (x)| < 7.5 \cdot 10^{-8}$ and the coefficients $b_i$ are fixed (see implementation).

```cpp
inline const double b0 = 0.2316419;
inline const double b1 = 0.319381530;
inline const double b2 = -0.356563782;
inline const double b3 = 1.781477937;
inline const double b4 = -1.821255978;
inline const double b5 = 1.330274429;
```


### References
* Hagan, P. S. et al. *Implied volatility formulas for Heston models.* Wilmott 2018.98 (2018)
* Bartlett, B. *Hedging Under SABR Model.* Wilmott Magazine, (2006)
* Oblój, J. *Fine-Tune Your Smile: Correction to Hagan et al*. preprint arXiv:0708.0998 (2007)
* Paulot, L. *Asymptotic implied volatility at the second order with application to the SABR model.* In Large Deviations and Asymptotic Methods in Finance (pp. 37-69). Springer, Cham. (2015)
* Henry-Labordère, P. *A general asymptotic implied volatility for stochastic volatility 
models.* http://arxiv.org/abs/cond-mat/0504317 (2005)